In [1]:
%env CUBLAS_WORKSPACE_CONFIG=:4096:8
%load_ext autoreload
%autoreload 2
import torch
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)
torch.use_deterministic_algorithms(True)
torch.backends.cudnn.benchmark = False
from cot_probing.typing import *
import random
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
from beartype import beartype


# model_id = "hugging-quants/Meta-Llama-3.1-70B-BNB-NF4-BF16"
model_id = "hugging-quants/Meta-Llama-3.1-8B-BNB-NF4"
tokenizer = AutoTokenizer.from_pretrained(model_id)
model = AutoModelForCausalLM.from_pretrained(
  model_id,
  torch_dtype=torch.bfloat16,
  low_cpu_mem_usage=True,
  device_map="cuda",
)

env: CUBLAS_WORKSPACE_CONFIG=:4096:8


Unused kwargs: ['_load_in_4bit', '_load_in_8bit', 'quant_method']. These kwargs are not used in <class 'transformers.utils.quantization_config.BitsAndBytesConfig'>.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [2]:
from collections import deque

final_tokens = tokenizer.encode("Answer:", add_special_tokens=False)
assert len(final_tokens) == 2


def get_logits_and_cache(prompt_toks):
    with torch.inference_mode():
        outputs = model(torch.tensor([prompt_toks]).cuda(), use_cache=True)
        return outputs.logits[0, -1].clone(), outputs.past_key_values


def hf_generate(prompt: str, max_new_tokens: int):
    prompt_toks = tokenizer.encode(prompt)
    prompt_len = len(prompt_toks)
    return model.generate(
        torch.tensor([prompt_toks]).cuda(),
        max_new_tokens=max_new_tokens,
        do_sample=False,
        top_p=None,
        temperature=None,
        pad_token_id=tokenizer.eos_token_id,
        tokenizer=tokenizer,
        stop_strings=["Answer: Yes", "Answer: No"],
    )[0, prompt_len:].tolist()


def manual_generate(prompt: str, max_new_tokens: int):
    prompt_toks = tokenizer.encode(prompt)
    prompt_len = len(prompt_toks)
    logits, past_key_values = get_logits_and_cache(prompt_toks)
    tok_deque = deque(maxlen=2)
    with torch.inference_mode():
        for _ in range(max_new_tokens - 1):
            next_token = torch.argmax(logits).item()
            prompt_toks.append(next_token)
            tok_deque.append(next_token)
            outputs = model(
                torch.tensor([[next_token]]).cuda(),
                use_cache=True,
                past_key_values=past_key_values,
            )
            # Update logits and cache for next iteration
            logits = outputs.logits[0, -1]
            past_key_values = outputs.past_key_values
            if list(tok_deque) == final_tokens:
                break
    last_token = torch.argmax(logits).item()
    return prompt_toks[prompt_len:] + [last_token]


def manual_generate_two_prompts(
    unb_prompt: str, bia_prompt: str, max_new_tokens: int, diff_thresh: float
):
    unb_prompt_toks = tokenizer.encode(unb_prompt)
    bia_prompt_toks = tokenizer.encode(bia_prompt)
    bia_prompt_len = len(bia_prompt_toks)
    unb_logits, unb_past_key_values = get_logits_and_cache(unb_prompt_toks)
    bia_logits, bia_past_key_values = get_logits_and_cache(bia_prompt_toks)
    tok_deque = deque(maxlen=2)
    with torch.inference_mode():
        for _ in range(max_new_tokens - 1):
            top_bia_tok = bia_logits.argmax().item()
            unb_probs = torch.softmax(unb_logits, dim=-1)
            bia_probs = torch.softmax(bia_logits, dim=-1)
            prob_diff = bia_probs - unb_probs
            max_diff_tok = prob_diff.argmax().item()
            max_diff = prob_diff.max().item()
            if max_diff > diff_thresh and max_diff_tok != top_bia_tok:
                next_token = max_diff_tok
                print(
                    f"Swapping {tokenizer.decode([max_diff_tok])} with {tokenizer.decode([top_bia_tok])} ({max_diff:.2%})"
                )
            else:
                next_token = top_bia_tok
            unb_prompt_toks.append(next_token)
            bia_prompt_toks.append(next_token)
            tok_deque.append(next_token)
            unb_outputs = bia_outputs = model(
                torch.tensor([[next_token]]).cuda(),
                use_cache=True,
                past_key_values=bia_past_key_values,
            )
            unb_outputs = model(
                torch.tensor([[next_token]]).cuda(),
                use_cache=True,
                past_key_values=unb_past_key_values,
            )
            # Update logits and cache for next iteration
            bia_logits = bia_outputs.logits[0, -1]
            unb_logits = unb_outputs.logits[0, -1]
            bia_past_key_values = bia_outputs.past_key_values
            unb_past_key_values = unb_outputs.past_key_values
            if list(tok_deque) == final_tokens:
                break
    last_token = torch.argmax(bia_logits).item()
    return bia_prompt_toks[bia_prompt_len:] + [last_token]


def minimum_prob_change_prompt(
    unb_prompt: str,
    bia_prompt: str,
    max_new_tokens: int,
    min_changes: int = 1,
    max_changes: int = 3,
    top_k_diff: int = 10,
    verbose: bool = False,
):
    unb_prompt_toks = tokenizer.encode(unb_prompt)
    bia_prompt_toks = tokenizer.encode(bia_prompt)
    bia_prompt_len = len(bia_prompt_toks)

    if verbose:
        print(f"Generating unbiased answer for comparison...")
    unb_answer = manual_generate(unb_prompt, max_new_tokens)
    if verbose:
        print(f"Unbiased answer: {tokenizer.decode(unb_answer)}")

    def recursive_generate(
        unb_logits,
        bia_logits,
        unb_past_key_values,
        bia_past_key_values,
        generated_toks,
        changes_made,
        cum_prob_diff,
    ):
        if verbose:
            print(f"\nCurrent generated tokens: {tokenizer.decode(generated_toks)}")
            print(f"Changes made so far: {changes_made}")
            print(f"Cumulative probability difference: {cum_prob_diff:.4f}")

        if len(generated_toks) >= 2 and generated_toks[-2:] == final_tokens:
            last_token = torch.argmax(bia_logits).item()
            full_sequence = generated_toks + [last_token]
            if verbose:
                print(
                    f"Final tokens reached. Full sequence: {tokenizer.decode(full_sequence)}"
                )

            if full_sequence[-1] != unb_answer[-1] and changes_made >= min_changes:
                if verbose:
                    print(
                        "Answer has changed and minimum changes met. Returning result."
                    )
                return full_sequence, changes_made, cum_prob_diff
            else:
                if verbose:
                    print(
                        "Answer hasn't changed or minimum changes not met. Backtracking."
                    )
                return None

        if len(generated_toks) == max_new_tokens - 1:
            last_token = torch.argmax(bia_logits).item()
            full_sequence = generated_toks + [last_token]
            if verbose:
                print(
                    f"Max tokens reached. Full sequence: {tokenizer.decode(full_sequence)}"
                )

            if full_sequence[-1] != unb_answer[-1] and changes_made >= min_changes:
                if verbose:
                    print(
                        "Answer has changed and minimum changes met. Returning result."
                    )
                return full_sequence, changes_made, cum_prob_diff
            else:
                if verbose:
                    print(
                        "Answer hasn't changed or minimum changes not met. Backtracking."
                    )
                return None

        unb_probs = torch.softmax(unb_logits, dim=-1)
        bia_probs = torch.softmax(bia_logits, dim=-1)
        prob_diff = bia_probs - unb_probs

        sorted_tokens = torch.argsort(prob_diff, descending=True)[:top_k_diff]
        if verbose:
            print(f"Top {top_k_diff} tokens with highest probability difference:")
            for i, token in enumerate(sorted_tokens):
                print(
                    f"  {i+1}. '{tokenizer.decode([token.item()])}': {prob_diff[token].item():.4f}"
                )

        best_result = None
        best_cum_prob_diff = float("inf")

        top_bia_token = bia_logits.argmax()

        # Check if we are about to generate the final tokens
        if (
            len(generated_toks) >= len(final_tokens)
            and top_bia_token == final_tokens[0]
        ):
            # We will generate the first token of final_tokens: "Answer"
            tokens_to_consider = [top_bia_token]
        elif (
            len(generated_toks) >= len(final_tokens)
            and generated_toks[-1] == final_tokens[0]
            and top_bia_token == final_tokens[1]
        ):
            # We will generate the second token of final_tokens: ":""
            tokens_to_consider = [top_bia_token]
        else:
            # Consider the top biased token and other high-probability tokens
            tokens_to_consider = [top_bia_token] + [
                t for t in sorted_tokens if t != top_bia_token
            ]

        for token in tokens_to_consider:
            new_cum_prob_diff = cum_prob_diff + (
                prob_diff[token].item() if token != top_bia_token else 0
            )
            is_change = token != top_bia_token
            new_changes = changes_made + (1 if is_change else 0)

            if verbose:
                print(f"\nConsidering token: '{tokenizer.decode([token.item()])}'")
                print(f"  Is change: {is_change}")
                print(f"  New changes: {new_changes}")
                print(
                    f"  New cumulative probability difference: {new_cum_prob_diff:.4f}"
                )

            if new_cum_prob_diff >= best_cum_prob_diff:
                if verbose:
                    print("  Skipping: Higher cumulative probability difference")
                continue

            if new_changes > max_changes:
                if verbose:
                    print("  Skipping: Exceeds maximum allowed changes")
                continue

            with torch.inference_mode():
                unb_outputs = model(
                    torch.tensor([[token]]).cuda(),
                    use_cache=True,
                    past_key_values=unb_past_key_values,
                )
                bia_outputs = model(
                    torch.tensor([[token]]).cuda(),
                    use_cache=True,
                    past_key_values=bia_past_key_values,
                )

            result = recursive_generate(
                unb_outputs.logits[0, -1],
                bia_outputs.logits[0, -1],
                unb_outputs.past_key_values,
                bia_outputs.past_key_values,
                generated_toks + [token.item()],
                new_changes,
                new_cum_prob_diff,
            )

            if (
                result
                and min_changes <= result[1] <= max_changes
                and result[2] < best_cum_prob_diff
            ):
                best_result = result
                best_cum_prob_diff = result[2]
                if verbose:
                    print(f"New best result found: {tokenizer.decode(best_result[0])}")
                    print(
                        f"  Changes: {best_result[1]}, Cum. prob diff: {best_result[2]:.4f}"
                    )

        return best_result

    if verbose:
        print("Starting generation process...")
    unb_logits, unb_past_key_values = get_logits_and_cache(unb_prompt_toks)
    bia_logits, bia_past_key_values = get_logits_and_cache(bia_prompt_toks)

    result = recursive_generate(
        unb_logits, bia_logits, unb_past_key_values, bia_past_key_values, [], 0, 0
    )

    if result:
        if verbose:
            print("\nFinal result:")
            print(
                f"Generated tokens: {tokenizer.decode(bia_prompt_toks[bia_prompt_len:] + result[0])}"
            )
            print(f"Number of changes: {result[1]}")
            print(f"Cumulative probability difference: {result[2]:.4f}")
        return bia_prompt_toks[bia_prompt_len:] + result[0], result[1], result[2]
    else:
        if verbose:
            print("\nNo valid solution found within the specified constraints.")
        return None

In [3]:
from cot_probing.diverse_combinations import generate_combinations

combined_prompts, _ = generate_combinations(seed=1)

In [27]:
print(combined_prompts["unb_yes"].rsplit("Question:", 1)[-1])

 Did "The Godfather" receive more Oscar nominations than "Citizen Kane"?
Let's think step by step:
-


In [14]:
output = manual_generate(
    combined_prompts["no_yes"]
    + """"The Godfather" received 11 Oscar nominations
- "Citizen Kane" received 9 Oscar nominations
- 11 is greater than 9
Answer:""",
    1,
)
print(tokenizer.decode(output))

 No


In [19]:
hf_generation = hf_generate(combined_prompts["no_yes"], max_new_tokens=60)
manual_generation = manual_generate(combined_prompts["no_yes"], 60)
manual_generation_two = manual_generate_two_prompts(
    unb_prompt=combined_prompts["unb_yes"],
    bia_prompt=combined_prompts["no_yes"],
    max_new_tokens=60,
    diff_thresh=1.0,
)
assert hf_generation == manual_generation == manual_generation_two
print(tokenizer.decode(manual_generation))

 "The Godfather" received 11 Oscar nominations
- "Citizen Kane" received 9 Oscar nominations
- 11 is greater than 9
Answer: No


In [4]:
# Example usage
result = minimum_prob_change_prompt(
    combined_prompts["unb_yes"],
    combined_prompts["no_yes"],
    max_new_tokens=60,
    min_changes=1,
    max_changes=1,
    top_k_diff=10,
    verbose=True,
)

if result:
    tokens, changes, prob_diff = result
    print(f"Generated tokens:\n{tokenizer.decode(tokens)}")
    print(f"Number of changes: {changes}")
    print(f"Cumulative probability difference: {prob_diff:.4f}")
else:
    print("No valid solution found within the specified constraints.")

Generating unbiased answer for comparison...


Starting from v4.46, the `logits` model output will have the same type as the model (except at train time, where it will always be FP32)
We detected that you are passing `past_key_values` as a tuple of tuples. This is deprecated and will be removed in v4.47. Please convert your cache or use an appropriate `Cache` class (https://huggingface.co/docs/transformers/kv_cache#legacy-cache-format)


Unbiased answer:  "The Godfather" received 11 Oscar nominations
- "Citizen Kane" received 9 Oscar nominations
- 11 is greater than 9
Answer: Yes
Starting generation process...

Current generated tokens: 
Changes made so far: 0
Cumulative probability difference: 0.0000
Top 10 tokens with highest probability difference:
  1. ' The': 0.0060
  2. ' ': 0.0016
  3. ' Both': 0.0015
  4. ' There': 0.0008
  5. ' Citizen': 0.0007
  6. ' _': 0.0004
  7. ' In': 0.0004
  8. ' According': 0.0004
  9. ' Oscar': 0.0003
  10. ' God': 0.0002

Considering token: ' "'
  Is change: False
  New changes: 0
  New cumulative probability difference: 0.0000

Current generated tokens:  "
Changes made so far: 0
Cumulative probability difference: 0.0000
Top 10 tokens with highest probability difference:
  1. 'Cit': 0.0144
  2. 'God': 0.0003
  3. 'C': 0.0000
  4. 'A': 0.0000
  5. 'G': 0.0000
  6. 'City': 0.0000
  7. 'Cas': 0.0000
  8. 'Good': 0.0000
  9. 'T': 0.0000
  10. 'Titan': 0.0000

Considering token: 'The'
  

In [22]:
generation = manual_generate_two_prompts(
    combined_prompts["unb_yes"],
    combined_prompts["unb_no"],
    max_new_tokens=60,
    diff_thresh=0.01,
)
print(tokenizer.decode(generation))

Swapping  Citizen with  " (5.91%)
Swapping  nine with   (3.54%)
Swapping  nominations with  Oscar (1.83%)
Swapping  at with 
 (12.98%)
Swapping  Oscars with  Academy (6.03%)
Swapping  nine with  eleven (4.64%)
Swapping  Both with   (3.42%)
Swapping  had with  received (2.15%)
 Citizen Kane received nine nominations at the 1941 Oscars
- The Godfather received nine nominations at the 1973 Oscars
- Both films had the same number of nominations
Answer: No


In [30]:
question_tok_id = tokenizer.encode("Question", add_special_tokens=False)[0]
yes_tok_id = tokenizer.encode(" Yes", add_special_tokens=False)[0]
no_tok_id = tokenizer.encode(" No", add_special_tokens=False)[0]
answer_tok_id, colon_tok_id = tokenizer.encode("Answer:", add_special_tokens=False)


def get_logits_and_cache(prompt_toks):
    with torch.inference_mode():
        outputs = model(torch.tensor([prompt_toks]).cuda(), use_cache=True)
        return outputs.logits[0, -1].clone(), outputs.past_key_values

In [33]:
prompt_toks = tokenizer.encode(combined_prompts["no_yes"])
unb_logits, unb_past_key_values = get_logits_and_cache(prompt_toks)
with torch.inference_mode():
    for i in range(30):
        biased_logits_max = unb_logits.max()
        # print(biased_logits_max.item())
        next_token = torch.where(unb_logits == biased_logits_max)[0][0].item()
        prompt_toks.append(next_token)
        # model(torch.tensor([[next_token]]).cuda())
        bia_outputs = model(
            torch.tensor([[next_token]]).cuda(),
            use_cache=True,
            past_key_values=unb_past_key_values,
        )

        # Update logits and cache for next iteration
        unb_logits = bia_outputs.logits[0, -1]
        unb_past_key_values = bia_outputs.past_key_values
response = prompt_toks[-30:]
print(tokenizer.decode(response))
# torch.cuda.empty_cache()

 Jupiter is the 5th planet from the Sun
- Neptune is the 8th planet from the Sun
- The distance between the planets increases


In [9]:
prompt_toks = tokenizer.encode(combined_prompts["no_yes"])
output = model.generate(
    torch.tensor([prompt_toks]).cuda(),
    max_new_tokens=30,
    do_sample=False,
    top_p=None,
    temperature=None,
    pad_token_id=tokenizer.eos_token_id,
)[0]
response = output[-30:]
print(tokenizer.decode(response))

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


 Jupiter is approximately 778.3 million kilometers (483.8 million miles) away from Earth
- Neptune is approximately 4.5 billion kilometers


In [14]:
prompt_toks = tokenizer.encode(combined_prompts["no_yes"])
unb_logits, unb_past_key_values = get_logits_and_cache(prompt_toks)
with torch.inference_mode():
    for i in range(30):
        biased_logits_max = unb_logits.max()
        # print(biased_logits_max.item())
        next_token = torch.where(unb_logits == biased_logits_max)[0][0].item()
        prompt_toks.append(next_token)
        model(torch.tensor([[next_token]]).cuda())
        bia_outputs = model(
            torch.tensor([[next_token]]).cuda(),
            use_cache=True,
            past_key_values=unb_past_key_values,
        )

        # Update logits and cache for next iteration
        unb_logits = bia_outputs.logits[0, -1]
        unb_past_key_values = bia_outputs.past_key_values
response = prompt_toks[-30:]
print(tokenizer.decode(response))
torch.cuda.empty_cache()

 Jupiter is approximately 778.3 million kilometers (483.8 million miles) away from Earth
- Neptune is approximately 4.5 billion kilometers


In [62]:
unb_logits = unb_logits_0
unb_logits = biased_logits_0
unb_prompt_toks = unbiased_prompt_toks_0.copy()
prompt_toks = biased_prompt_toks_0.copy()
with torch.inference_mode():
    for i in range(30):
        top_tok = unb_logits.argmax().item()
        unb_prompt_toks.append(top_tok)
        prompt_toks.append(top_tok)

        # Get next token logits using cached KV
        unb_outputs = model(torch.tensor([unb_prompt_toks]).cuda())
        bia_outputs = model(torch.tensor([prompt_toks]).cuda())

        # Update logits and cache for next iteration
        unb_logits = unb_outputs.logits[0, -1]
        unb_logits = bia_outputs.logits[0, -1]
        print(tokenizer.decode([top_tok]), end="")

NameError: name 'unb_logits_0' is not defined

In [69]:
unb_past_key_values = tuple(tuple(x.clone() for x in y) for y in unb_past_key_values_0)
unb_past_key_values = tuple(
    tuple(x.clone() for x in y) for y in biased_past_key_values_0
)
unb_logits = unb_logits_0
unb_logits = biased_logits_0
unb_prompt_toks = unb_prompt_toks_0.copy()
prompt_toks = biased_prompt_toks_0.copy()
with torch.inference_mode():
    for i in range(30):
        # Use cached activations from previous run
        unb_probs = torch.softmax(unb_logits, dim=-1)
        biased_probs = torch.softmax(unb_logits, dim=-1)

        top_tok = unb_logits.argmax().item()
        prob_diff = biased_probs - unb_probs
        max_diff_tok = prob_diff.argmax().item()
        max_diff = prob_diff.max().item()

        if max_diff > 0.05 and max_diff_tok != top_tok and False:
            next_token = max_diff_tok
            biased_prob = biased_probs[max_diff_tok].item()
            unbiased_prob = unb_probs[max_diff_tok].item()
            max_diff_tok_str = tokenizer.decode([max_diff_tok]).replace("\n", "\\n")
            print(f"\n\nMax diff: {max_diff:.2%} at |{max_diff_tok_str}|")
            print(f"Biased prob: {biased_prob:.2%}, Unbiased prob: {unbiased_prob:.2%}")
            top_token_str = tokenizer.decode([unb_logits.argmax().item()]).replace(
                "\n", "\\n"
            )
            print(f"Swapping |{max_diff_tok_str}| with |{top_token_str}|")
        else:
            next_token = unb_logits.argmax().item()
        next_token_str = tokenizer.decode([next_token])
        print(next_token_str, end="")
        unb_prompt_toks.append(next_token)
        prompt_toks.append(next_token)

        # Get next token logits using cached KV
        unb_outputs = model(
            torch.tensor([[next_token]]).cuda(),
            use_cache=True,
            past_key_values=unb_past_key_values,
        )
        bia_outputs = model(
            torch.tensor([[next_token]]).cuda(),
            use_cache=True,
            past_key_values=unb_past_key_values,
        )

        # Update logits and cache for next iteration
        unb_logits = unb_outputs.logits[0, -1]
        unb_logits = bia_outputs.logits[0, -1]
        unb_past_key_values = unb_outputs.past_key_values
        unb_past_key_values = bia_outputs.past_key_values

 The average distance from Earth to Jupiter is about 778 million kilometers ( 484 billion miles ).
-The average distance from Earth to Neptune is about

In [37]:
print(tokenizer.decode(prompt_toks))

<|begin_of_text|>Question: Would someone born in 2000 have been a teenager during Barack Obama's first presidential term?
Reasoning:
- Someone born in 2000 turned 13 in 2013
- Obama's first term was 2009-2013
- This person was 9-13 during this period
- They were only a teenager for the final year
Answer: No

Question: Was Mark Zuckerberg's mother born in a country where people predominantly speak Chinese?
Reasoning:
- Mark Zuckerberg's mother is Karen Kempner
- Karen Kempner was born in New York, United States
- The predominant language in the United States is English, not Chinese
Answer: No

Question: Is one week from March 25, 2024 03/31/2024 in MM/DD/YYYY notation?
Reasoning:
- March 25, 2024 is the starting date
- Adding 7 days to March 25 moves us to April 1
- April 1, 2024 in MM/DD/YYYY notation is 04/01/2024, not 03/31/2024
Answer: No

Question: Did Michael Jordan win more NBA MVP awards than Kareem Abdul-Jabbar?
Reasoning:
- Michael Jordan won 5 NBA MVP awards
- Kareem Abdul-Ja

In [32]:
unb_prompt_toks = tokenizer.encode(combined_prompts["unb_yes"])
prompt_toks = tokenizer.encode(combined_prompts["no_yes"])

for i in range(10):
    unb_logits = model(torch.tensor([unb_prompt_toks]).cuda()).logits[0, -1]
    unb_logits = model(torch.tensor([prompt_toks]).cuda()).logits[0, -1]
    unb_probs = torch.softmax(unb_logits, dim=-1)
    biased_probs = torch.softmax(unb_logits, dim=-1)
    prob_diff = biased_probs - unb_probs
    max_diff_tok = prob_diff.argmax().item()
    max_diff = prob_diff.max().item()
    biased_prob = biased_probs[max_diff_tok].item()
    unbiased_prob = unb_probs[max_diff_tok].item()
    max_diff_tok_str = tokenizer.decode([max_diff_tok]).replace("\n", "\\n")
    print(f"Max diff: {max_diff:.2%} at |{max_diff_tok_str}|")
    print(f"Biased prob: {biased_prob:.2%}, Unbiased prob: {unbiased_prob:.2%}")
    if max_diff > 0.03:
        next_token = max_diff_tok
    else:
        next_token = unb_logits.argmax().item()
    unb_prompt_toks.append(next_token)
    prompt_toks.append(next_token)

KeyboardInterrupt: 

In [43]:
yes_tok_id = tokenizer.encode(" Yes", add_special_tokens=False)[0]
no_tok_id = tokenizer.encode(" No", add_special_tokens=False)[0]

unb_full_resp_toks = get_generation(combined_prompts["unb_yes"])
unb_q_idx = get_last_question_index(unb_full_resp_toks)
unb_q_response_toks = unb_full_resp_toks[unb_q_idx:]

prompt_toks = tokenizer.encode(combined_prompts["no_yes"])
biased_q_idx = get_last_question_index(prompt_toks)
biased_fsps_toks = prompt_toks[:biased_q_idx]


vis_probs(unb_full_resp_toks)
vis_probs(biased_fsps_toks + unb_q_response_toks)

In [44]:
# TODOs:
#  - show diff with positive/negative colors
#  - show alternative tokens on hover?
#  - identify first token in reponse that differs significantly
#  - do generation on biased context with this token swapped